# Blockchain API: Get data for a single address

In [1]:
import pandas as pd
import altair as alt

In [2]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 1000
pd.set_option("display.max_colwidth", None)

### Call url

In [3]:
url = 'https://blockchain.info/rawaddr/1Bm9Eqyd46qhJH8pQi7W8QxDA5VhbtKbXK?format=json&limit=1000'

### Read the results

In [8]:
src = pd.read_json(url)

### How many records? 

In [ ]:
len(src)

### The transaction specific data are in a nexted column called 'txs'

In [ ]:
txs = pd.json_normalize(src['txs'])

### Deal with dates, times

In [ ]:
txs['datetime'] = pd.to_datetime(txs['time'], unit='s')

In [ ]:
txs['date'] = txs['datetime'].dt.date
txs['time'] = txs['datetime'].dt.time

### Only include fields that seem descriptive

In [ ]:
txs_slim = txs[["datetime", 'date', 'time', 'result', 'balance', 'fee', 'hash', 'ver', 'vin_sz', 'vout_sz', 'size', 'weight', 'tx_index',
       'block_index', 'block_height', ]].copy()

In [ ]:
txs_slim.head()

---

### Aggregate

In [ ]:
txs_slim_grp = txs_slim.groupby(['date']).agg({'hash': 'count', 'fee':['mean', 'sum']}).round(2).reset_index()

In [ ]:
txs_slim_grp.columns = ['_'.join(col).strip() for col in txs_slim_grp.columns.values]

In [ ]:
txs_slim_grp.rename(columns={'date_':'date', 'hash_count': 'transaction_count'}, inplace=True)

In [ ]:
txs_slim_grp['date'] = pd.to_datetime(txs_slim_grp['date'])

---

### Average daily transaction count

In [ ]:
alt.Chart(txs_slim_grp).mark_bar().encode(
    x='date:T',
    y='transaction_count'
)

### Average daily transaction fee (not sure what this fee means yet)

In [ ]:
alt.Chart(txs_slim_grp).mark_bar().encode(
    x='date:T',
    y='fee_mean'
)

---

### Export

In [ ]:
txs_slim.to_csv('data/raw/transactions.csv', index=False)